In [1]:
import db_conn
import pandas as pd
import numpy as np
import copy
import statsmodels.formula.api as sm
import scipy.stats as st

In [2]:
# plt.style.use('dark_background')
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
sql = """SELECT * FROM score_line"""
score_line = db_conn.select_query(sql)
score_line_pd = pd.DataFrame(score_line, columns=['match_id', 'season_year', 'division', 'id', 'half_type', 'play_time', 'time_range', 'home_score', 'away_score', 'home_team_id', 'away_team_id', 'score_team_id', 'winning_team', 'score_player'])
score_line_pd = score_line_pd[score_line_pd.id > 0]
score_line_pd.head()

,match_id,season_year,division,id,half_type,play_time,time_range,home_score,away_score,home_team_id,away_team_id,score_team_id,winning_team,score_player
1,2013-1-001,2013,1,2,1,29,2,1,0,10,25,10,0,1
2,2013-1-001,2013,1,3,1,32,3,1,1,10,25,25,0,2
3,2013-1-001,2013,1,4,2,2,4,2,1,10,25,10,0,3
5,2013-1-001,2013,1,5,2,38,6,2,2,10,25,25,0,4
7,2013-1-002,2013,1,6,1,4,1,0,1,19,5,5,19,5


In [4]:
goal_type_pd = pd.DataFrame(score_line_pd, columns=['match_id', 'season_year', 'division', 'location', 'home_score', 'away_score', 'home_team_id', 'away_team_id', 'score_team_id', 'winning_team', 'score_player', 'g_fg', 'g_tg', 'g_og', 'g_lg', 'g_cg', 'winning_flag'])
goal_type_pd.location = np.where(goal_type_pd.home_team_id == goal_type_pd.score_team_id, 1, 0)
goal_type_pd.g_fg = np.where((goal_type_pd.home_score > 0) & (goal_type_pd.home_score + goal_type_pd.away_score == 1), True, False)
goal_type_pd.g_tg = np.where((goal_type_pd.home_score == goal_type_pd.away_score), True, False)
goal_type_pd.g_og = np.where(goal_type_pd.score_team_id == goal_type_pd.home_team_id, np.where(goal_type_pd.home_score - goal_type_pd.away_score == 1, True, False), np.where(goal_type_pd.away_score - goal_type_pd.home_score == 1, True, False))
goal_type_pd.g_og = np.where(goal_type_pd.g_fg, False, goal_type_pd.g_og)
goal_type_pd.g_lg = np.where(goal_type_pd.score_team_id == goal_type_pd.home_team_id, goal_type_pd.home_score > goal_type_pd.away_score, goal_type_pd.away_score > goal_type_pd.home_score)
goal_type_pd.g_lg = np.where(goal_type_pd.g_fg, False, goal_type_pd.g_lg)
goal_type_pd.g_lg = np.where(goal_type_pd.g_og, False, goal_type_pd.g_lg)
goal_type_pd.g_cg = np.where(goal_type_pd.score_team_id == goal_type_pd.home_team_id, np.where(goal_type_pd.home_score < goal_type_pd.away_score, True, False), np.where(goal_type_pd.away_score < goal_type_pd.home_score, True, False))
goal_type_pd.winning_flag = np.where(goal_type_pd.winning_team == 0, 0, np.where(goal_type_pd.home_team_id == goal_type_pd.winning_team, 1, 0))

goal_type_pd.head()

,match_id,season_year,division,location,home_score,away_score,home_team_id,away_team_id,score_team_id,winning_team,score_player,g_fg,g_tg,g_og,g_lg,g_cg,winning_flag
1,2013-1-001,2013,1,1,1,0,10,25,10,0,1,True,False,False,False,False,0
2,2013-1-001,2013,1,0,1,1,10,25,25,0,2,False,True,False,False,False,0
3,2013-1-001,2013,1,1,2,1,10,25,10,0,3,False,False,True,False,False,0
5,2013-1-001,2013,1,0,2,2,10,25,25,0,4,False,True,False,False,False,0
7,2013-1-002,2013,1,0,0,1,19,5,5,19,5,False,False,True,False,False,1


In [5]:
point_pre_pd = goal_type_pd.groupby(['match_id', 'location']).agg({'home_team_id': 'max', 'away_team_id': 'max', 'g_fg': 'sum', 'g_tg': 'sum', 'g_og': 'sum', 'winning_flag': 'max'}).reset_index()
point_pre_pd.winning_flag = np.where(point_pre_pd.location == 0, point_pre_pd.winning_flag * -1, point_pre_pd.winning_flag)
point_by_match_pd = pd.DataFrame(point_pre_pd, columns=['match_id', 'location', 'team_id', 'winning_flag', 'home_team_id', 'away_team_id', 'g_fg', 'g_tg', 'g_og', 'FGP', 'FTG', 'STG', 'FOG', 'SOG', 'TGP'])
point_by_match_pd.team_id = np.where(point_by_match_pd.location == 1, point_by_match_pd.home_team_id, point_by_match_pd.away_team_id)
point_by_match_pd = point_by_match_pd.drop(['home_team_id', 'away_team_id'], axis=1)
point_by_match_pd.head()

,match_id,location,team_id,winning_flag,g_fg,g_tg,g_og,FGP,FTG,STG,FOG,SOG,TGP
0,2013-1-001,0,25,0,False,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-1-001,1,10,0,True,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-1-002,0,5,-1,False,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-1-002,1,19,1,False,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-1-003,0,23,0,False,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
point_by_match_pd.FGP = point_by_match_pd.g_fg * 2
point_by_match_pd.FTG = np.where(point_by_match_pd.g_tg > 0, 0.44, 0)
point_by_match_pd.STG = np.where(point_by_match_pd.g_tg > 1, 0.6, 0)
point_by_match_pd.FOG = np.where(point_by_match_pd.g_og > 0, 0.7, 0)
point_by_match_pd.SOG = np.where(point_by_match_pd.g_og > 1, 0.88, 0)
point_by_match_pd.TGP = point_by_match_pd.FGP + point_by_match_pd.FTG + point_by_match_pd.STG + point_by_match_pd.FOG + point_by_match_pd.SOG
point_by_match_pd.head()

,match_id,location,team_id,winning_flag,g_fg,g_tg,g_og,FGP,FTG,STG,FOG,SOG,TGP
0,2013-1-001,0,25,0,False,2.0,0.0,0,0.44,0.6,0.0,0.0,1.04
1,2013-1-001,1,10,0,True,0.0,1.0,2,0.00,0.0,0.7,0.0,2.70
2,2013-1-002,0,5,-1,False,0.0,1.0,0,0.00,0.0,0.7,0.0,0.70
3,2013-1-002,1,19,1,False,1.0,1.0,0,0.44,0.0,0.7,0.0,1.14
4,2013-1-003,0,23,0,False,0.0,1.0,0,0.00,0.0,0.7,0.0,0.70


In [7]:
sql = """SELECT * FROM game_records"""
game_records = db_conn.select_query(sql)
game_records_pd = pd.DataFrame(game_records)
game_records_pd.head()

,away_team_id,away_team_score,division,game_date,game_id,game_stadium,game_time,home_team_id,home_team_score,season_year,winning_team
0,25,2,1,2013-03-02,2013-1-001,서울 월드컵,15:00,10,2,2013,0
1,5,1,1,2013-03-02,2013-1-002,울산 문수,14:45,19,2,2013,19
2,23,1,1,2013-03-02,2013-1-003,광양 전용,15:00,21,0,2013,23
3,13,2,1,2013-03-03,2013-1-004,탄천 종합,14:00,12,1,2013,13
4,2,0,1,2013-03-03,2013-1-005,인천 전용,14:00,20,0,2013,0


In [8]:
match_list_pd = pd.DataFrame(pd.concat([game_records_pd[['game_id', 'home_team_id']], game_records_pd[['game_id', 'away_team_id']]], ignore_index=True).sort_values('game_id').reset_index(drop=True), columns=['game_id', 'location', 'home_team_id', 'away_team_id', 'team_id'])
match_list_pd.location = np.where(match_list_pd.home_team_id.isna(), 0, 1)
match_list_pd.team_id = np.where(match_list_pd.home_team_id.isna(), match_list_pd.away_team_id, match_list_pd.home_team_id)
match_list_pd = match_list_pd.drop(['home_team_id', 'away_team_id'], axis=1)
match_list_pd.columns = ['match_id', 'location', 'team_id']
match_list_pd = match_list_pd.sort_values(['match_id', 'location'])
match_list_pd.head()

,match_id,location,team_id
1,2013-1-001,0,25.0
0,2013-1-001,1,10.0
2,2013-1-002,0,5.0
3,2013-1-002,1,19.0
5,2013-1-003,0,23.0


In [9]:
def set_winning_flag(record):
    if record.winning_flag in range(-1, 2):
        return record.winning_flag
        
    temp = full_point_by_match_pd[(full_point_by_match_pd.match_id == record.match_id) & (full_point_by_match_pd.winning_flag.notna())]
    if temp.__len__() == 1:
        return 0 if temp.winning_flag.iloc[0] == 0 else temp.winning_flag.iloc[0] * -1
    elif temp.__len__() == 0:
        return 0
    else:
        return record.winning_flag

In [10]:
full_point_by_match_pd = pd.DataFrame(pd.merge(match_list_pd, point_by_match_pd, how='left', on=['match_id', 'location', 'team_id']), columns=['match_id', 'season_year', 'division', 'location', 'team_id', 'winning_flag', 'g_fg', 'g_tg', 'g_og', 'FGP', 'FTG', 'STG', 'FOG', 'SOG', 'TGP'])
full_point_by_match_pd.g_fg = full_point_by_match_pd.g_fg.fillna(False)
full_point_by_match_pd.winning_flag = full_point_by_match_pd.apply(lambda x: set_winning_flag(x), axis=1)
full_point_by_match_pd.season_year = full_point_by_match_pd.match_id.apply(lambda x: int(x.split('-')[0]))
full_point_by_match_pd.division = full_point_by_match_pd.match_id.apply(lambda x: int(x.split('-')[1]))
full_point_by_match_pd = full_point_by_match_pd.fillna(0)
full_point_by_match_pd.head()

,match_id,season_year,division,location,team_id,winning_flag,g_fg,g_tg,g_og,FGP,FTG,STG,FOG,SOG,TGP
0,2013-1-001,2013,1,0,25.0,0.0,False,2.0,0.0,0.0,0.44,0.6,0.0,0.0,1.04
1,2013-1-001,2013,1,1,10.0,0.0,True,0.0,1.0,2.0,0.00,0.0,0.7,0.0,2.70
2,2013-1-002,2013,1,0,5.0,-1.0,False,0.0,1.0,0.0,0.00,0.0,0.7,0.0,0.70
3,2013-1-002,2013,1,1,19.0,1.0,False,1.0,1.0,0.0,0.44,0.0,0.7,0.0,1.14
4,2013-1-003,2013,1,0,23.0,0.0,False,0.0,1.0,0.0,0.00,0.0,0.7,0.0,0.70


In [11]:
goal_point_rank_pd = full_point_by_match_pd.groupby(['season_year', 'division', 'team_id']).agg({'match_id': 'count', 'FGP': 'sum', 'FTG': 'sum', 'STG': 'sum', 'FOG': 'sum', 'SOG': 'sum', 'TGP': 'sum'}).reset_index()
goal_point_rank_pd[(goal_point_rank_pd.season_year == 2017) & (goal_point_rank_pd.division == 1)].sort_values('TGP', ascending=False)

,season_year,division,team_id,match_id,FGP,FTG,STG,FOG,SOG,TGP
108,2017,1,23.0,38,28.0,3.52,0.6,9.1,0.00,41.22
107,2017,1,22.0,38,24.0,2.64,0.6,12.6,0.88,40.72
98,2017,1,1.0,38,18.0,5.28,0.6,11.2,3.52,38.60
103,2017,1,13.0,38,18.0,3.96,0.6,13.3,2.64,38.50
102,2017,1,10.0,38,18.0,3.96,1.2,11.2,3.52,37.88
104,2017,1,19.0,38,20.0,3.96,0.0,9.8,1.76,35.52
106,2017,1,21.0,38,22.0,3.96,0.6,7.7,0.88,35.14
100,2017,1,5.0,38,22.0,4.40,0.6,7.0,0.88,34.88
109,2017,1,25.0,38,16.0,6.60,0.6,10.5,0.88,34.58
101,2017,1,9.0,38,14.0,5.72,0.6,6.3,2.64,29.26


In [12]:
sql = """SELECT * FROM season_ranking"""
season_ranking = db_conn.select_query(sql)
season_ranking_pd = pd.DataFrame(season_ranking, columns=['year', 'division', 'rank', 'team_id', 'points', 'scored_goal_count', 'losed_goal_count', 'match_count', 'win_count', 'draw_count', 'lose_count'])
season_ranking_pd[season_ranking_pd.year == 2017]

,year,division,rank,team_id,points,scored_goal_count,losed_goal_count,match_count,win_count,draw_count,lose_count
98,2017,1,1,22,75,73,35,38,22,9,7
99,2017,1,2,23,66,60,37,38,19,9,10
100,2017,1,3,13,64,63,41,38,17,13,8
101,2017,1,4,19,62,42,45,38,17,11,10
102,2017,1,5,10,61,56,42,38,16,13,9
103,2017,1,6,1,49,59,65,38,13,10,15
104,2017,1,7,25,52,64,60,38,15,7,16
105,2017,1,8,5,47,50,52,38,11,14,13
106,2017,1,9,20,39,32,53,38,7,18,13
107,2017,1,10,21,35,53,69,38,8,11,19


In [16]:
temp = pd.DataFrame(pd.merge(goal_point_rank_pd, season_ranking_pd, how='left', left_on=['season_year', 'division', 'team_id'], right_on=['year', 'division', 'team_id']), columns = ['season_year', 'division', 'team_id', 'match_id', 'FGP', 'FTG', 'STG', 'FOG', 'SOG', 'TGP', 'custom_rank', 'year', 'rank', 'points', 'scored_goal_count', 'losed_goal_count', 'match_count', 'win_count', 'draw_count', 'lose_count', 'rank_diff'])
temp.columns = columns = ['season_year', 'division', 'team_id', 'match_id', 'FGP', 'FTG', 'STG', 'FOG', 'SOG', 'TGP', 'custom_rank', 'year', 'real_rank', 'points', 'scored_goal_count', 'losed_goal_count', 'match_count', 'win_count', 'draw_count', 'lose_count', 'rank_diff']
                    
temp.custom_rank = temp.groupby(['season_year', 'division']).TGP.rank(ascending=False)
temp.rank_diff = abs(temp.custom_rank - temp.real_rank)
temp = temp[['season_year', 'division', 'team_id', 'match_id', 'FGP', 'FTG', 'STG', 'FOG', 'SOG', 'TGP', 'custom_rank', 'real_rank', 'rank_diff', 'points', 'scored_goal_count', 'losed_goal_count', 'match_count', 'win_count', 'draw_count', 'lose_count']]
# temp[(temp.season_year == 2017) & (temp.division == 1)].sort_values('TGP', ascending=False)

display(temp.groupby(['season_year', 'division']).agg({'rank_diff': ['sum', 'mean']}))
# display(temp[(temp.season_year == 2017) & (temp.division == 1)].sort_values(['season_year', 'scored_goal_count'], ascending=[True, False]))
display(temp[temp.rank_diff > 2].sort_values(['season_year', 'division', 'real_rank']).__len__())

rank_diff          
                           sum      mean
season_year division                    
2013        1             20.0  1.428571
            2              5.0  0.714286
            3              1.0  0.500000
2014        1             22.0  1.833333
            2             11.0  1.222222
            3              0.0  0.000000
2015        1             18.0  1.500000
            2             17.0  1.700000
            3              0.0  0.000000
2016        1             36.0  3.000000
            2             17.0  1.700000
            3              1.0  0.500000
2017        1             18.0  1.500000
            2              8.0  0.800000
            3              1.0  0.500000

24

In [14]:
temp[(temp.season_year == 2017) & (temp.division == 1)][['FGP', 'FTG', 'STG', 'FOG', 'SOG', 'rank_diff']].corr()

,FGP,FTG,STG,FOG,SOG,rank_diff
FGP,1.000000,-0.674405,0.142857,0.282675,-0.484131,-0.243599
FTG,-0.674405,1.000000,0.012043,-0.319147,0.176236,0.205356
STG,0.142857,0.012043,1.000000,0.353343,0.374101,-0.568399
FOG,0.282675,-0.319147,0.353343,1.000000,0.396558,-0.143457
SOG,-0.484131,0.176236,0.374101,0.396558,1.000000,-0.062541
rank_diff,-0.243599,0.205356,-0.568399,-0.143457,-0.062541,1.000000
